<a href="https://colab.research.google.com/github/jrojasquiroz/msprov/blob/main/1-limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza de datos y cálculos previos

#### ¿Qué hacemos en este cuaderno?
Ordenar la información en bruto que INEI envió por correo. Esta corresponde principalmente a límites de las *ciudades principales*.
Lo que haremos es calcular la población de cada parte de los polígonos urbanos de las *ciudades principales*.

#### ¿Qué procedimiento seguimos?
Básicamente, intersecciones entre distritos y polígonos urbanos.

#### ¿Qué datos obtenemos?
1. Un gdf que tiene solo las manchas urbanas repartidas por distritos (*ciudades_interseccion.parquet*). Aquí si nos interesa su geometría.
2. Un gdf que tiene partidas las manchas urbanas por distritos y la población que la conforma (*ciudades_pob.parquet*). No nos interesa su geometría, sino sus datos.

<mark>Ojo</mark>: Los archivos *.parquet* pueden ser leídos por cualquier software SIG sin problema.

Primero, cargamos las librerías necesarias.

In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np

A continuación, defino mi directorio de trabajo. Si te interesa reproducir este trabajo, debes adecuarlo a la ruta que lleve hacia tu carpeta de Google Drive (en caso trabajes en Google Colab) o hacia la de tu PC (en caso trabajes en algún IDE como los Jupyter Notebooks).

In [2]:
os.chdir('/content/drive/MyDrive/Documentos personales/1-Investigaciones y análisis/Perú una país de provincias/revisión 1')

El siguiente paso será realizar una intersección entre los polígonos urbanos (`pu`) y los distritos (`dist`) para saber cómo están distribuidos.

In [3]:
pu=gpd.read_parquet('rawdata/pu-principalesciudades-og_WGS84.parquet')
dist=gpd.read_parquet('rawdata/lim-dist_WGS84.parquet')

In [4]:
pu=pu.to_crs('EPSG:3857') #Pseudo-Mercator
dist=dist.to_crs('EPSG:3857')

In [5]:
inter=pu.overlay(dist, how='intersection')
inter=inter[['CIUDAD','NOMBDIST','CODUBIGEO','IDPROV','NOMBPROV','geometry']] #Nos quedamos solo con las columnas que nos interesan
inter['ID'] = inter.index + 1

Guardo el archivo generado.

In [6]:
# Ruta del archivo GeoParquet comprimido de salida
output_parquet = 'data/ciudades_interseccion.parquet'

# Guardar como GeoParquet con compresión gzip
inter.to_parquet(output_parquet, compression='gzip', index=False)

Ahora vamos a obtener el total de población por manzana. Para ello usamos el archivo vectorial de las manzanas de las ciudades del país (`gdf`) y la tabla que tiene información de la cantidad de personas que habitan en cada una (`pob`).

In [7]:
gdf=gpd.read_parquet('rawdata/mz-principalesciudades-edit_WGS84.parquet')
pob=pd.read_excel('rawdata/Personas x manzana - INEI.xlsx')

gdf=gdf.to_crs('EPSG:3857')

Cambiamos los nombres de las columnas para que coincidan

In [8]:
gdf = gdf.rename(columns={'Mz_Edit':'Mz'})
pob = pob.rename(columns={'Cod':'Mz'})

Unimos `gdf` con `pob` y lo convertimos en centroides para que pese menos el archivo.

In [9]:
## Hacemos el merge
gdf = gdf.merge(pob, on='Mz')

In [10]:
## Convertimos el gdf de manzanas en centroides
centros=gpd.GeoDataFrame(gdf.centroid)

In [11]:
centros.head()

,0
0,POINT (-8669524.377 -693808.991)
1,POINT (-8669371.073 -694019.822)
2,POINT (-8669378.964 -694067.903)
3,POINT (-8669350.380 -694165.716)
4,POINT (-8669293.824 -694264.816)


In [12]:
centros.rename(columns={0:'geometry'},
                inplace=True)

In [13]:
# Establece la columna 'geometry' como la columna de geometría activa
centros = centros.set_geometry('geometry')

# Ahora puedes establecer el CRS
centros = centros.set_crs('EPSG:3857')

In [14]:
## Ahora los centros deben 'absorber' los datos de su gdf original
mz = gpd.sjoin(centros, gdf, how="left", op='intersects')

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [15]:
mz.head()

,geometry,index_right,Mz_Original,Mz,CIUDAD,UBIGEO,CODPROV,CODDPTO,NOMBDIST,NOMBPROV,NOMBDPTO,Población
0,POINT (-8669524.377 -693808.991),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (-8669371.073 -694019.822),1.0,01010100100001K,010101000100100001K,CHACHAPOYAS,010101,01,01,CHACHAPOYAS,CHACHAPOYAS,AMAZONAS,70.0
2,POINT (-8669378.964 -694067.903),2.0,01010100100001L,010101000100100001L,CHACHAPOYAS,010101,01,01,CHACHAPOYAS,CHACHAPOYAS,AMAZONAS,62.0
3,POINT (-8669350.380 -694165.716),3.0,01010100100002D,010101000100100002D,CHACHAPOYAS,010101,01,01,CHACHAPOYAS,CHACHAPOYAS,AMAZONAS,42.0
4,POINT (-8669293.824 -694264.816),4.0,01010100100003B,010101000100100003B,CHACHAPOYAS,010101,01,01,CHACHAPOYAS,CHACHAPOYAS,AMAZONAS,70.0


Nos quedamos con las columnas que nos interesan.

In [16]:
mz=mz[['geometry','Mz','UBIGEO','Población']]

Hacemos el spatial join de los centroides de las manzanas (que tienen info de población - `mz`) con las manchas urbanas distribuidas por 'pedacitos' (la intersección del inicio - `inter`).

In [17]:
intento1=gpd.sjoin(mz,inter,how="left", op="within")   #por defecto, 'sjoin' utiliza la opción 'intersect'
                                                         #sin embargo, al usar 'within' los resultados pueden ser hasta
                                                         #10 veces más rápidos
intento1.head()

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


,geometry,Mz,UBIGEO,Población,index_right,CIUDAD,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,ID
0,POINT (-8669524.377 -693808.991),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (-8669371.073 -694019.822),010101000100100001K,010101,70.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
2,POINT (-8669378.964 -694067.903),010101000100100001L,010101,62.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
3,POINT (-8669350.380 -694165.716),010101000100100002D,010101,42.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
4,POINT (-8669293.824 -694264.816),010101000100100003B,010101,70.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0


In [18]:
len(intento1)

116375

In [19]:
#3.Filtramos solo aquellas columnas que estén dentro de las manchas urbanas de las ciudades principales
filtro=intento1['ID']>=0
intento2 = intento1[filtro]

intento2.head()

,geometry,Mz,UBIGEO,Población,index_right,CIUDAD,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,ID
1,POINT (-8669371.073 -694019.822),010101000100100001K,010101,70.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
2,POINT (-8669378.964 -694067.903),010101000100100001L,010101,62.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
3,POINT (-8669350.380 -694165.716),010101000100100002D,010101,42.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
4,POINT (-8669293.824 -694264.816),010101000100100003B,010101,70.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0
5,POINT (-8669341.131 -694221.467),010101000100100005,010101,45.0,47.0,CHACHAPOYAS,CHACHAPOYAS,10101.0,0101,CHACHAPOYAS,48.0


Guardo el archivo obtenido.

In [20]:
# Ruta del archivo GeoParquet comprimido de salida
output_parquet = 'data/ciudades_pob.parquet'

# Guardar como GeoParquet con compresión gzip
intento2.to_parquet(output_parquet, compression='gzip', index=False)

In [21]:
len(intento2)

116323